In [10]:
import os

# os.chdir(os.path.dirname(__file__))
os.chdir(os.path.abspath('..'))
print(os.getcwd())

In [61]:
from utils.mano import ManoLayer
from utils.utils import load_ho_meta

mano_layer = ManoLayer()
anno = load_ho_meta()

mano_hand = mano_layer(anno)

/Users/shugo/workspace/rendering/utils/utils.py:18: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [31]:
#https://github.com/otaheri/MANO/blob/5869ab059c1bf31cc724f57eaf93e041135e8960/mano/joints_info.py#L17C1-L25C2
TIP_IDS = {
    'mano': {
            'thumb':            744,
            'index':            320,
            'middle':           443,
            'ring':                 554,
            'pinky':            671,
        }
}

In [62]:
import plotly.graph_objects as go
from utils.utils import add_mesh_to_fig

fig = go.Figure()
add_mesh_to_fig(fig, mano_hand.vertices[0], mano_layer.faces, opacity=0.5)
fig.add_trace(
    go.Scatter3d(
        x=mano_hand.joints[0][:, 0],
        y=mano_hand.joints[0][:, 1],
        z=mano_hand.joints[0][:, 2],
        mode='markers+text',
        marker=dict(
            size=2,
            color='black',
            # line=dict(color='black', width=1),
        ),
        text=[f"{i:02d}" for i in range(len(mano_hand.joints[0]))],
        opacity=1,
    )
)
for k, v in TIP_IDS['mano'].items():
    fig.add_trace(
        go.Scatter3d(
            x=mano_hand.vertices[0][v:v+1, 0],
            y=mano_hand.vertices[0][v:v+1, 1],
            z=mano_hand.vertices[0][v:v+1, 2],
            mode='markers+text',
            marker=dict(
                size=2,
                color='green',
            ),
            text=[f"{k}"],
            opacity=1,
        ),
    )
fig.update_layout(scene=dict(camera=dict(projection=dict(type="orthographic"))))
fig.show()

In [85]:
from tqdm import tqdm
import numpy as np
import torch
from vctoolkit import Timer



def model_run(params):
    params = torch.tensor(params, dtype=torch.float32)
    global_orient = params[:3].unsqueeze(0)
    hand_pose = params[3:3+45].unsqueeze(0)
    betas = params[3+45:3+45+10].unsqueeze(0)
    # betas = None
    transl = params[3+45+10:3+45+10+3].unsqueeze(0)
    model_output = mano_layer.mano_layer['right'](
        global_orient=global_orient, hand_pose=hand_pose, betas=betas, transl=transl)
    return model_output.joints[0].detach().numpy()

class Solver:
    def __init__(self, eps=1e-5, max_iter=30, mse_threshold=1e-8, verbose=False):
        """
        Parameters
        ----------
        eps : float, optional
        Epsilon for derivative computation, by default 1e-5
        max_iter : int, optional
        Max iterations, by default 30
        mse_threshold : float, optional
        Early top when mse change is smaller than this threshold, by default 1e-8
        verbose : bool, optional
        Print information in each iteration, by default False
        """
        self.eps = eps
        self.max_iter = max_iter
        self.mse_threshold = mse_threshold
        self.verbose = verbose
        self.timer = Timer()

    def get_derivative(self, model, params, n):
        """
        Compute the derivative by adding and subtracting epsilon

        Parameters
        ----------
        model : object
        Model wrapper to be manipulated.
        params : np.ndarray
        Current model parameters.
        n : int
        The index of parameter.

        Returns
        -------
        np.ndarray
        Derivative with respect to the n-th parameter.
        """
        params1 = np.array(params)
        params2 = np.array(params)

        params1[n] += self.eps
        params2[n] -= self.eps

        res1 = model(params1)
        res2 = model(params2)

        d = (res1 - res2) / (2 * self.eps)

        return d.ravel()

    def solve(self, model, target, init=None, u=1e-3, v=1.5):
        """
        Solver for the target.

        Parameters
        ----------
        model : object
        Wrapper to be manipulated.
        target : np.ndarray
        Optimization target.
        init : np,ndarray, optional
        Initial parameters, by default None
        u : float, optional
        LM algorithm parameter, by default 1e-3
        v : float, optional
        LM algorithm parameter, by default 1.5

        Returns
        -------
        np.ndarray
        Solved model parameters.
        """
        if init is None:
            init = np.zeros(61)
        out_n = np.shape(model(init).ravel())[0]
        jacobian = np.zeros([out_n, init.shape[0]])

        last_update = 0
        last_mse = 0
        params = init
        for i in tqdm(range(self.max_iter)):
            residual = (model(params) - target).reshape(out_n, 1)
            mse = np.mean(np.square(residual))

            if abs(mse - last_mse) < self.mse_threshold:
                return params

            for k in range(params.shape[0]):
                jacobian[:, k] = self.get_derivative(model, params, k)

            jtj = np.matmul(jacobian.T, jacobian)
            jtj = jtj + u * np.eye(jtj.shape[0])

            update = last_mse - mse
            delta = np.matmul(
                np.matmul(np.linalg.inv(jtj), jacobian.T), residual
            ).ravel()
            params -= delta

            if update > last_update and update > 0:
                u /= v
            else:
                u *= v

            last_update = update
            last_mse = mse

            if self.verbose:
                print(i, self.timer.tic(), mse)

        return params

init = np.concatenate([
    mano_hand.global_orient.reshape(-1).detach().numpy(),
    np.zeros(45),
    np.zeros(10),
    np.zeros(3),
])
# init = None

solver = Solver(verbose=True)
params_solved = solver.solve(
    model_run, 
    mano_hand.joints.numpy(), # - mano_hand.joints.numpy()[:, 0:1],
    init=init)

  7%|▋         | 2/30 [00:00<00:03,  9.11it/s]

0 0.05364489555358887 0.06803136
1 0.17402005195617676 8.497232e-06


 13%|█▎        | 4/30 [00:00<00:03,  6.60it/s]

2 0.20416688919067383 3.4616623e-06
3 0.15643906593322754 2.279238e-06


 20%|██        | 6/30 [00:00<00:03,  7.14it/s]

...

6 0.09654116630554199 1.667992e-06
7 0.10314488410949707 1.6165532e-06


 30%|███       | 9/30 [00:01<00:02,  8.50it/s]

8 0.10461115837097168 1.5803116e-06


 37%|███▋      | 11/30 [00:01<00:02,  6.93it/s]

9 0.29741787910461426 1.5591959e-06
10 0.12614989280700684 1.5434038e-06


In [86]:
import plotly.graph_objects as go
from utils.utils import add_mesh_to_fig

with torch.no_grad():
    params = torch.tensor(params_solved, dtype=torch.float32)
    global_orient = params[:3].unsqueeze(0)
    hand_pose = params[3:3+45].unsqueeze(0)
    betas = params[3+45:3+45+10].unsqueeze(0)
    transl = params[3+45+10:3+45+10+3].unsqueeze(0)
    mano_solved = mano_layer.mano_layer['right'](global_orient=global_orient, hand_pose=hand_pose, betas=betas, transl=transl)

    fig = go.Figure()
    add_mesh_to_fig(fig, mano_hand.vertices[0], mano_layer.faces, opacity=0.5, color='blue')
    fig.add_trace(
        go.Scatter3d(
            x=mano_hand.joints[0][:, 0],
            y=mano_hand.joints[0][:, 1],
            z=mano_hand.joints[0][:, 2],
            mode='markers',
            marker=dict(
                size=2,
                color='black',
                # line=dict(color='black', width=1),
            ),
            text=[f"{i:02d}" for i in range(len(mano_hand.joints[0]))],
            opacity=1,
        )
    )
    add_mesh_to_fig(fig, mano_solved.vertices[0], mano_layer.faces, opacity=0.5, color='red')
    fig.add_trace(
        go.Scatter3d(
            x=mano_solved.joints[0][:, 0],
            y=mano_solved.joints[0][:, 1],
            z=mano_solved.joints[0][:, 2],
            mode='markers',
            marker=dict(
                size=2,
                color='black',
                # line=dict(color='black', width=1),
            ),
            text=[f"{i:02d}" for i in range(len(mano_solved.joints[0]))],
            opacity=1,
        )
    )
    for k, v in TIP_IDS['mano'].items():
        fig.add_trace(
            go.Scatter3d(
                x=mano_solved.vertices[0][v:v+1, 0],
                y=mano_solved.vertices[0][v:v+1, 1],
                z=mano_solved.vertices[0][v:v+1, 2],
                mode='markers',
                marker=dict(
                    size=2,
                    color='green',
                ),
                text=[f"{k}"],
                opacity=1,
            ),
        )
    fig.update_layout(scene=dict(camera=dict(projection=dict(type="orthographic"))))
    fig.show()